In [1]:
""" 
Optim package - T Martin 10 May 2020
This code if for multivariate optimizer using surrogate response surface created by several ML models
The next target point to study is either based on minimum point found, either by using a surrogate if r2 value of 
a model is above a certain threshold
"""

' \nOptim package - T Martin 10 May 2020\nThis code if for multivariate optimizer using surrogate response surface created by several ML models\nThe next target point to study is either based on minimum point found, either by using a surrogate if r2 value of \na model is above a certain threshold\n'

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Oct 16 10:52:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.86       Driver Version: 430.86       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P3200 wi... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   57C    P8     5W /  N/A |    740MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [4]:
import lightgbm
import plotly.express as px

In [5]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [6]:
# If we want to export in html in a separate browser
#mport plotly.io as pio
#pio.renderers.default = "browser" 

In [13]:
# Define function to minimize
# x is a vector containing what we want to evaluate
def funct(x):
    x1=x[0]
    x2=x[1]
    #f=x1*x1+x1*x2+2*np.sin((x1+x2)*2)+4*np.cos(x2*2)
    #f=x1*x1+x2*x2-x1*x2+20*np.sin(x1-x2)-8*np.cos(x2*2)
    #f=(x1-3)*(x1-3)+(x2-1)*(x2-1)+x1*x2
    f=(x1-3)*(x2-3)+(x2-1)*(x2-1)+x1*x2
    #f=x[0]**2+x[1]**2-x[2]
    #launch exothermia
    #err1=....
    #err2-....
    #f=err1^2+0.5*err2^2

    return f

In [14]:
# Here we define the range for the xi we want to find. It is the amount of variables
lower_limit=[-5,-5]
upper_limit=[5,5]
boundaries_=[lower_limit, upper_limit]

In [15]:
def Optimtm(eval_function, boundaries, plot_steps=False):
  # This function adds the points to the matrix
  # The matrix is a dataframe
  def add_matrix(X,points):
      df=points
      for i in range(len(df)):
          variables=df.iloc[i]
          variables['z']=funct(variables)
          X=X.append(variables)
      return X
  # Define function providing N randoms points in the restraied domain 
  def next_point(current_space,n=5):
      next_space=current_space
      pt=pd.DataFrame()
      for i in range(n):
          vecteur=current_space['Center']
          Radius=current_space['Radius']
          new_vec=vecteur+Radius*np.random.choice((-1,1),size=len(Radius))*(np.random.random(len(Radius)))
          for j in range(len(new_vec)):
              if new_vec[j]<space.iloc[j]['Center']-space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']-space.iloc[j]['Radius']
              if new_vec[j]>space.iloc[j]['Center']+space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']+space.iloc[j]['Radius']                              
          pt['n'+str(i)]=new_vec
      pt=pt.T
      pt.columns=X.drop('z',axis=1).columns
      #next_space['Radius']=next_space['Radius']*epsilon   <------ A vérifier!!
      return pt
  def training_loop(X,domain,AI=False,n=5,epsilon=0.95):
      df=next_point(domain,n)
      Matrix=add_matrix(X,df)
      x=Matrix.drop('z',axis=1).values
      y=Matrix['z'].values
      if AI==True:
          # Need to do a CV split with Train test split
          models=[LinearRegression(), lightgbm.LGBMRegressor(),GaussianProcessRegressor(random_state=0),XGBRegressor(objective='reg:squarederror')]
          # Additional models: MLPRegressor(hidden_layer_sizes=(5,3),max_iter=300, verbose=False,learning_rate_init=0.05),

          #X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.8,shuffle=True)
          r2max=-100
          best_model=models[1]
          for model_loop in models:
            score=cross_val_score(model_loop,x,y,cv=5)
            r2=np.mean(score)     
            if r2>r2max:
              r2max=r2
              best_model=model_loop 
          if r2max<0.6:
            #print('R2 insufficient %0.2f'%r2max)
            theor_min=np.argmin(X.z.values)
            lowest_value=np.min(X.z.values)
            minimum=X.iloc[theor_min]
          else:
              model=best_model
              print(type(model).__name__, " has been found with r2=%0.2f"%r2max)
              # Retrain the model with all data
              model.fit(x,y)
              # Predict and find the minimum in one shot
              Prediction=model.predict(srf_x.values)
              lowest_value_ai_predicted=np.min(Prediction)
              minimum=srf.iloc[np.argmin(Prediction)]    
              # Evaluate at minimum
              lowest_value=funct(minimum.values)  
      else:
          theor_min=np.argmin(X.z.values)
          lowest_value=np.min(X.z.values)
          minimum=X.iloc[theor_min]
      minimum=minimum.drop('z')
      # New space should be centered around this minimum
      domain['Center']=(minimum.values*0.5+domain.Center*0.5)
      domain['Radius']=domain['Radius']*epsilon
      return Matrix,domain,lowest_value

  nvar=len(boundaries[0]) #How many variables we want to optimize
  # Initialize the space domain. Instead of [-5,5] we convert to Center + Radius [0,5]
  space=pd.DataFrame(columns=['Center','Radius'])
  for i in np.arange(0,nvar):
    space.loc[i]=[(boundaries[0][i]+boundaries[1][i])/2,0.5*(boundaries[1][i]-boundaries[0][i])]
  # Initialize domain
  domain=space.copy()

  # Initialize X matrix. It will contain all evaluated points. We start with 5 evaluations points:
  X=pd.DataFrame()
  for i in np.arange(0,nvar):
      X['x'+str(i)]=0
     # Fill X with 5 points picked at domain boundary
  for i in np.arange(0,nvar):
    X['x'+str(i)]=[boundaries[0][i],boundaries[0][i],boundaries[1][i],boundaries[1][i],(boundaries[0][i]+boundaries[1][i])/2,(boundaries[0][i]+boundaries[1][i])/2,(boundaries[0][i]+boundaries[1][i])/2,boundaries[0][i],boundaries[1][i]]
  X['z']=0
     # Evaluate the points  
  Z=[]
  for i in np.arange(0,len(X)):
      variables=X.drop(['z'],axis=1).iloc[i]
      #variables['target']=funct(variables[['x0','x1']].values)
      z=eval_function(variables.values)
      Z.append(z)
  X['z']=Z

  # Initialize Evaluation surface. We generate N points for evaluating the ML model. N can be big (1000)
  N=5000
  srf=pd.DataFrame()
  loc=0
  # We will sample 100 points from the boundary. This 100 gives the granularity.
  for i in np.arange(0,nvar):
    xi=np.random.choice(np.linspace(boundaries[0][i],boundaries[1][i],num=100),N)
    srf['x'+str(i)]=xi
  Z=[]
  for i in np.arange(0,len(srf)):
    z=funct(srf.iloc[i].values)
    Z.append(z)
  srf['z']=Z
  # for japan srf=DF
  srf_x=srf.drop(['z'],axis=1)
  # Plot evaluation surface
  plot_srf=True
  if plot_srf==True:
    fig = px.scatter_3d(srf, x='x0', y='x1', z='z',color='z')
    #fig.show()
  print("Minimum found on evaluation surface : ")
  index=np.argmin(srf.z)
  print(srf.iloc[index])
 
  # Run the training loop
  N_loops=20
  Max_points=20
  point_amount=Max_points
  Dom=[] #pd.DataFrame(columns=['x0 C','x0 R','x1 C','x1 R'])]
  for i in np.arange(0,N_loops):
      # Point amount should decrease. At the beginning, it should be 10. At the end, 2.
      point_amount=np.int((Max_points-3)-(Max_points-3)*i/N)+3

      X,domain,minimum=training_loop(X,domain,AI=True,n=point_amount,epsilon=0.8)
      minimum=np.min(X.z.values)
      #print("epoch ",i," Minimum=%0.2f"%minimum)
  min_found=np.min(X.z.values)
  best_point=X.iloc[np.argmin(X.z.values)]
  if plot_srf==True:
    fig.update_traces(mode='markers', marker_line_width=2, marker_size=5)
    fig2=px.scatter_3d(X,x='x0',y='x1',z='z',color_discrete_sequence=['red'])
    fig2.update_traces(mode='markers', marker_line_width=2, marker_size=8)
    fig3=px.scatter_3d(pd.DataFrame(best_point).T,x='x0',y='x1',z='z',color_discrete_sequence=['green'])
    fig3.update_traces(mode='markers', marker_line_width=2, marker_size=15)
    fig.add_trace(fig2.data[0])
    fig.add_trace(fig3.data[0])
    fig.show()
  return X,min_found, best_point

In [16]:
X,minimum, best_point=Optimtm(funct,boundaries_,True)
print(best_point)

Minimum found on evaluation surface : 
x0    -5.000000
x1     4.898990
z    -24.484746
Name: 920, dtype: float64
[10:59:10] src/objective/objective.cc:21: Objective candidate: binary:hinge
[10:59:10] src/objective/objective.cc:21: Objective candidate: multi:softmax
[10:59:10] src/objective/objective.cc:21: Objective candidate: multi:softprob
[10:59:10] src/objective/objective.cc:21: Objective candidate: rank:pairwise
[10:59:10] src/objective/objective.cc:21: Objective candidate: rank:ndcg
[10:59:10] src/objective/objective.cc:21: Objective candidate: rank:map
[10:59:10] src/objective/objective.cc:21: Objective candidate: reg:linear
[10:59:10] src/objective/objective.cc:21: Objective candidate: reg:logistic
[10:59:10] src/objective/objective.cc:21: Objective candidate: binary:logistic
[10:59:10] src/objective/objective.cc:21: Objective candidate: binary:logitraw
[10:59:10] src/objective/objective.cc:21: Objective candidate: count:poisson
[10:59:10] src/objective/objective.cc:21: Objecti

C:\Users\TMA4076\AppData\Local\conda\conda\envs\datascience\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning:


The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.

C:\Users\TMA4076\AppData\Local\conda\conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: b'[10:59:10] src/objective/objective.cc:23: Unknown objective function reg:squarederror'




[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:hinge
[10:59:11] src/objective/objective.cc:21: Objective candidate: multi:softmax
[10:59:11] src/objective/objective.cc:21: Objective candidate: multi:softprob
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:pairwise
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:ndcg
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:map
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:linear
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:logistic
[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:logistic
[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:logitraw
[10:59:11] src/objective/objective.cc:21: Objective candidate: count:poisson
[10:59:11] src/objective/objective.cc:21: Objective candidate: survival:cox
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:gamma
[10:59:11] sr

C:\Users\TMA4076\AppData\Local\conda\conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: b'[10:59:11] src/objective/objective.cc:23: Unknown objective function reg:squarederror'




[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:hinge
[10:59:11] src/objective/objective.cc:21: Objective candidate: multi:softmax
[10:59:11] src/objective/objective.cc:21: Objective candidate: multi:softprob
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:pairwise
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:ndcg
[10:59:11] src/objective/objective.cc:21: Objective candidate: rank:map
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:linear
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:logistic
[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:logistic
[10:59:11] src/objective/objective.cc:21: Objective candidate: binary:logitraw
[10:59:11] src/objective/objective.cc:21: Objective candidate: count:poisson
[10:59:11] src/objective/objective.cc:21: Objective candidate: survival:cox
[10:59:11] src/objective/objective.cc:21: Objective candidate: reg:gamma
[10:59:11] sr

...

[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:hinge
[10:59:16] src/objective/objective.cc:21: Objective candidate: multi:softmax
[10:59:16] src/objective/objective.cc:21: Objective candidate: multi:softprob
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:pairwise
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:ndcg
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:map
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:linear
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:logistic
[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:logistic
[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:logitraw
[10:59:16] src/objective/objective.cc:21: Objective candidate: count:poisson
[10:59:16] src/objective/objective.cc:21: Objective candidate: survival:cox
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:gamma
[10:59:16] sr

C:\Users\TMA4076\AppData\Local\conda\conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: b'[10:59:16] src/objective/objective.cc:23: Unknown objective function reg:squarederror'




[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:hinge
[10:59:16] src/objective/objective.cc:21: Objective candidate: multi:softmax
[10:59:16] src/objective/objective.cc:21: Objective candidate: multi:softprob
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:pairwise
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:ndcg
[10:59:16] src/objective/objective.cc:21: Objective candidate: rank:map
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:linear
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:logistic
[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:logistic
[10:59:16] src/objective/objective.cc:21: Objective candidate: binary:logitraw
[10:59:16] src/objective/objective.cc:21: Objective candidate: count:poisson
[10:59:16] src/objective/objective.cc:21: Objective candidate: survival:cox
[10:59:16] src/objective/objective.cc:21: Objective candidate: reg:gamma
[10:59:16] sr

C:\Users\TMA4076\AppData\Local\conda\conda\envs\datascience\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning:

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: b'[10:59:16] src/objective/objective.cc:23: Unknown objective function reg:squarederror'




x0    -5.0
x1     5.0
z    -25.0
Name: n2, dtype: float64


In [11]:
pd.DataFrame(best_point).T

,x0,x1,z
n1,3.349889,-0.667409,0.666929


In [30]:
# Print Minimum found
min=np.min(X.z.values)
print("Minimum found:")
print(X.iloc[np.argmin(X.z.values)])
#print("Theoretical Minimum : ")
#print(srf.iloc[np.argmin(srf.z)])


Minimum found:
x0    2.324348
x1   -0.131932
z     1.431119
Name: n6, dtype: float64
